In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

2025-09-06 22:12:32.106713: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv('Churn_Modelling.csv')

data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

label_encoder = LabelEncoder()
data["Gender"] = label_encoder.fit_transform(data["Gender"])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [3]:
one_hot_encoder = OneHotEncoder(drop='first')
geo_encoded = one_hot_encoder.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,1.0,0.0


In [4]:
X = data.drop(columns=["Exited"])
Y = data["Exited"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
def create_model(neurons=32, layers=1):
      model = Sequential()
      model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
      
      for _ in range(layers - 1):
          model.add(Dense(neurons, activation='relu'))
      
      model.add(Dense(1, activation='sigmoid'))
      model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
      return model

In [10]:
model=KerasClassifier(model=create_model, epochs=50, batch_size=32, verbose=0)

In [14]:
param_grid = {
      'model__neurons': [16, 32, 64],
      'layers': [1],
      'epochs': [50, 100],
      'batch_size': [16, 32, 64],      
}

In [16]:
# ...existing code...

model = KerasClassifier(
    model=create_model,
    epochs=50,
    batch_size=32,
    verbose=0,
    layers=1  # Default value, will be overridden by param_grid
)

param_grid = {
    'model__neurons': [16, 32, 64],
    'model__layers': [1, 2, 3],
    'epochs': [50, 100],
    'batch_size': [16, 32, 64]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_result = grid.fit(X_train, Y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
# ...existing code...

Fitting 3 folds for each of 54 candidates, totalling 162 fits


2025-09-06 22:26:16.352545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-06 22:26:16.612742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-06 22:26:16.673422: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-06 22:26:16.696956: I tensorflow/core/platform/cpu_featu

[CV] END batch_size=16, epochs=50, model__layers=1, model__neurons=32; total time= 1.1min


/home/hi-born/miniconda3/envs/genai/lib/python3.10/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END batch_size=16, epochs=50, model__layers=1, model__neurons=16; total time= 1.1min


...

[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=16; total time=  44.4s


/home/hi-born/miniconda3/envs/genai/lib/python3.10/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=16; total time=  45.6s
[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=16; total time=  44.7s
[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=32; total time=  45.3s
[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=32; total time=  42.4s
[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=32; total time=  45.0s
[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=64; total time=  33.1s
[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=64; total time=  29.0s
[CV] END batch_size=64, epochs=100, model__layers=3, model__neurons=64; total time=  26.9s


/home/hi-born/miniconda3/envs/genai/lib/python3.10/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1757179130.747225 1309031 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Best: 0.8584991673490175 using {'batch_size': 32, 'epochs': 50, 'model__layers': 1, 'model__neurons': 16}
